In [1]:
#!pip install tqdm
import torchvision.models as models
import sys
import torch
from torch import nn
import time
from tqdm import tqdm
import torch.optim as optim
import copy

# Specify where to find the data preparation class
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [2]:
# VGG16 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
DIMENSIONS = 3
#SIZE = 256

In [3]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="train", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader
test_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="test", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([5, 2, 5, 3, 6])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([1, 3, 4, 3, 4])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 5, 0, 6, 3])


In [4]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
vgg19 = models.vgg19(pretrained=True)


C:\Users\qo2707es\Downloads\Facial_Emotion_Detec-main\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\qo2707es\Downloads\Facial_Emotion_Detec-main\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\qo2707es/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:08<00:00, 66.1MB/s] 


In [5]:
for param in vgg19.parameters():
    param.requires_grad = False
dataloaders = {'train':train_data_loader, 'val':test_data_loader}


In [6]:
num_classes = len(train_data_loader.dataset.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer_vgg19 = optim.Adam(vgg19.parameters())
num_epochs = 20
for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch,num_epochs -1))
    print('-'*10)
    total_batch = len(train_data_loader.dataset) // 16
    for phase in ['train', 'val']:
        if phase == 'train':
            vgg19.train()
        else:
            vgg19.eval()

        running_loss = 0.0
        corrects = 0

        for i,(inputs, labels) in enumerate(dataloaders[phase]):
            optimizer_vgg19.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs_vgg19 = vgg19(inputs)
                _, preds_vgg19 = torch.max(outputs_vgg19, 1)

                loss_vgg19 = criterion(outputs_vgg19, labels)

                if phase == 'train':
                    loss_vgg19.backward()
                    optimizer_vgg19.step()

                    if (i+1) % 5 == 0:
                        print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                             %(epoch+1, num_epochs, i+1, total_batch, loss_vgg19.item()))

            running_loss += loss_vgg19.item() * inputs.size(0)
            corrects += torch.sum(preds_vgg19 == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase])
        epoch_acc = corrects.double() / len(dataloaders[phase])

        print('{} Loss: {:.4f} | Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0/19
----------
Epoch [1/20], lter [5/1752] Loss: 1.7404
Epoch [1/20], lter [10/1752] Loss: 1.8892
Epoch [1/20], lter [15/1752] Loss: 1.7805
Epoch [1/20], lter [20/1752] Loss: 1.7202
Epoch [1/20], lter [25/1752] Loss: 1.9876
Epoch [1/20], lter [30/1752] Loss: 1.6650
Epoch [1/20], lter [35/1752] Loss: 2.0754
Epoch [1/20], lter [40/1752] Loss: 1.9165
Epoch [1/20], lter [45/1752] Loss: 1.6169
Epoch [1/20], lter [50/1752] Loss: 1.7655
Epoch [1/20], lter [55/1752] Loss: 1.9691
Epoch [1/20], lter [60/1752] Loss: 2.0244
Epoch [1/20], lter [65/1752] Loss: 1.7025
Epoch [1/20], lter [70/1752] Loss: 1.7305
Epoch [1/20], lter [75/1752] Loss: 1.6628
Epoch [1/20], lter [80/1752] Loss: 1.5790
Epoch [1/20], lter [85/1752] Loss: 1.8091
Epoch [1/20], lter [90/1752] Loss: 1.6063
Epoch [1/20], lter [95/1752] Loss: 2.0564
Epoch [1/20], lter [100/1752] Loss: 1.6162
Epoch [1/20], lter [105/1752] Loss: 1.3688
Epoch [1/20], lter [110/1752] Loss: 1.5911
Epoch [1/20], lter [115/1752] Loss: 2.0607
Epoch [1/

  5%|▌         | 1/20 [46:59<14:52:56, 2819.83s/it]

val Loss: 25.0896 | Acc: 6.2517
Epoch 1/19
----------
Epoch [2/20], lter [5/1752] Loss: 2.3332
Epoch [2/20], lter [10/1752] Loss: 1.2583
Epoch [2/20], lter [15/1752] Loss: 1.4643
Epoch [2/20], lter [20/1752] Loss: 1.5326
Epoch [2/20], lter [25/1752] Loss: 1.8774
Epoch [2/20], lter [30/1752] Loss: 1.6724
Epoch [2/20], lter [35/1752] Loss: 1.9542
Epoch [2/20], lter [40/1752] Loss: 1.4930
Epoch [2/20], lter [45/1752] Loss: 1.9861
Epoch [2/20], lter [50/1752] Loss: 1.5940
Epoch [2/20], lter [55/1752] Loss: 1.5622
Epoch [2/20], lter [60/1752] Loss: 1.3380
Epoch [2/20], lter [65/1752] Loss: 1.8366
Epoch [2/20], lter [70/1752] Loss: 1.9196
Epoch [2/20], lter [75/1752] Loss: 1.6689
Epoch [2/20], lter [80/1752] Loss: 2.1099
Epoch [2/20], lter [85/1752] Loss: 1.5367
Epoch [2/20], lter [90/1752] Loss: 2.0449
Epoch [2/20], lter [95/1752] Loss: 2.1086
Epoch [2/20], lter [100/1752] Loss: 1.6082
Epoch [2/20], lter [105/1752] Loss: 1.9485
Epoch [2/20], lter [110/1752] Loss: 1.3953
Epoch [2/20], lter [

 10%|█         | 2/20 [1:31:22<13:38:08, 2727.14s/it]

...

val Loss: 24.5958 | Acc: 6.4766
Epoch 18/19
----------
Epoch [19/20], lter [5/1752] Loss: 1.9016
Epoch [19/20], lter [10/1752] Loss: 1.2389
Epoch [19/20], lter [15/1752] Loss: 1.6211
Epoch [19/20], lter [20/1752] Loss: 1.4775
Epoch [19/20], lter [25/1752] Loss: 1.5814
Epoch [19/20], lter [30/1752] Loss: 1.9059
Epoch [19/20], lter [35/1752] Loss: 1.7770
Epoch [19/20], lter [40/1752] Loss: 1.7667
Epoch [19/20], lter [45/1752] Loss: 1.5946
Epoch [19/20], lter [50/1752] Loss: 2.1111
Epoch [19/20], lter [55/1752] Loss: 2.4178
Epoch [19/20], lter [60/1752] Loss: 1.8100
Epoch [19/20], lter [65/1752] Loss: 1.8055
Epoch [19/20], lter [70/1752] Loss: 1.6993
Epoch [19/20], lter [75/1752] Loss: 2.1280
Epoch [19/20], lter [80/1752] Loss: 1.1811
Epoch [19/20], lter [85/1752] Loss: 2.0730
Epoch [19/20], lter [90/1752] Loss: 1.4002
Epoch [19/20], lter [95/1752] Loss: 1.8364
Epoch [19/20], lter [100/1752] Loss: 1.7828
Epoch [19/20], lter [105/1752] Loss: 1.5555
Epoch [19/20], lter [110/1752] Loss: 1.69

 95%|█████████▌| 19/20 [14:09:45<44:36, 2676.63s/it]  

val Loss: 25.3929 | Acc: 6.2517
Epoch 19/19
----------
Epoch [20/20], lter [5/1752] Loss: 1.5092
Epoch [20/20], lter [10/1752] Loss: 1.4972
Epoch [20/20], lter [15/1752] Loss: 1.3422
Epoch [20/20], lter [20/1752] Loss: 1.6794
Epoch [20/20], lter [25/1752] Loss: 1.3173
Epoch [20/20], lter [30/1752] Loss: 1.6274
Epoch [20/20], lter [35/1752] Loss: 2.0150
Epoch [20/20], lter [40/1752] Loss: 1.8394
Epoch [20/20], lter [45/1752] Loss: 2.3826
Epoch [20/20], lter [50/1752] Loss: 1.7964
Epoch [20/20], lter [55/1752] Loss: 1.4706
Epoch [20/20], lter [60/1752] Loss: 1.2780
Epoch [20/20], lter [65/1752] Loss: 1.9174
Epoch [20/20], lter [70/1752] Loss: 1.7568
Epoch [20/20], lter [75/1752] Loss: 1.1667
Epoch [20/20], lter [80/1752] Loss: 1.4190
Epoch [20/20], lter [85/1752] Loss: 1.7162
Epoch [20/20], lter [90/1752] Loss: 1.9485
Epoch [20/20], lter [95/1752] Loss: 1.4107
Epoch [20/20], lter [100/1752] Loss: 1.8034
Epoch [20/20], lter [105/1752] Loss: 1.1865
Epoch [20/20], lter [110/1752] Loss: 2.00

100%|██████████| 20/20 [14:54:16<00:00, 2682.83s/it]

val Loss: 25.8229 | Acc: 6.0490


In [7]:
# Saving the model
torch.save(vgg19.state_dict(), 'trained_vgg19.pt')

In [9]:
# Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['val']:
    # Forward pass
    inputs, target = batch_data
    output = vgg19(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 37.16%
